In [6]:
import h5py
import json
import numpy as np
import os
import pandas as pd

from reV.handlers.exclusions import ExclusionLayers
from reVX.handlers.geotiff import Geotiff
from reVX.utilities.exclusions_converter import ExclusionsConverter
from reVX.utilities.loggers import setup_logger

data_dir = os.path.join('/Users/mrossol/Git_Repos/reVX/tests/data/ri_exclusions')

layers = [os.path.join(data_dir, f) for f in os.listdir(data_dir)
          if f.startswith('ri') and f.endswith('.tif')]
excl_h5 = '/Users/mrossol/Git_Repos/reV/tests/data/ri_exclusions/ri_exclusions.h5'

logger = setup_logger('reVX', log_level='DEBUG')

In [7]:
with Geotiff(layers[1]) as f:
    tiff_profile = f.profile
    tiff_meta = f.meta
    
tiff_profile
tiff_crs = {k: v for k, v in [i.split("=") for i in tiff_profile['crs'].split(' ')]}

In [8]:
with ExclusionLayers(excl_h5) as f:
    print(f.layers)


['ri_padus', 'ri_reeds_regions', 'ri_smod', 'ri_srtm_slope']


In [9]:
excl = ExclusionsConverter(excl_h5)
out_path = '/Users/mrossol/Downloads/padus.tif'
profile, values = excl['ri_padus', out_path]

DEBUG - 2019-10-17 14:42:52,888 [exclusions_converter.py:307] : 	 - Extracting ri_padus from ri_exclusions.h5
DEBUG - 2019-10-17 14:42:52,901 [exclusions_converter.py:313] : 	- Writing ri_padus to /Users/mrossol/Downloads/padus.tif
DEBUG - 2019-10-17 14:42:53,130 [exclusions_converter.py:280] : 	- /Users/mrossol/Downloads/padus.tif created


In [4]:
excl = ExclusionsConverter(excl_h5)
#excl[os.path.basename(layers[0]).split('.')[0]] = layers[0]
ExclusionsConverter.create_h5(excl_h5, layers)

DEBUG - 2019-10-17 12:40:16,149 [exclusions_converter.py:128] : 	- Initializing /Users/mrossol/Git_Repos/reV/tests/data/ri_exclusions/ri_exclusions.h5 from /Users/mrossol/Git_Repos/reV/tests/data/ri_exclusions/ri_reeds_regions.tif
DEBUG - 2019-10-17 12:40:16,530 [exclusions_converter.py:134] : 	- "profile", "meta", and "shape" extracted from /Users/mrossol/Git_Repos/reV/tests/data/ri_exclusions/ri_reeds_regions.tif
DEBUG - 2019-10-17 12:40:16,533 [exclusions_converter.py:139] : 	- Default profile:
{'driver': 'GTiff', 'dtype': 'float64', 'nodata': 0.0, 'width': 972, 'height': 1434, 'count': 1, 'crs': '+proj=tmerc +lat_0=41.08333333333334 +lon_0=-71.5 +k=0.99999375 +x_0=100000 +y_0=0 +ellps=GRS80 +units=m +no_defs=True', 'transform': (90.0, 0.0, 65848.61752782026, 0.0, -90.0, 103948.14381277534), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
DEBUG - 2019-10-17 12:40:16,536 [exclusions_converter.py:141] : 	- Default shape:
(1434, 972)
DEBUG 